# Hyperelasticity
Author: Jørgen S. Dokken and Garth N. Wells

This section shows how to solve the hyperelasticity problem for deformation of a beam.

We will also show how to create a constant boundary condition for a vector function space.

We start by importing DOLFINx and some additional dependencies.
Then, we create a slender cantilever consisting of hexahedral elements and create the function space `V` for our unknown.

In [1]:
import numpy as np
import ufl

from petsc4py import PETSc
from mpi4py import MPI
from dolfinx import fem, mesh, plot
L = 20.0
domain = mesh.create_box(MPI.COMM_WORLD,[[0.0,0.0,0.0], [L, 1, 1]], [20, 5, 5], mesh.CellType.hexahedron)
V = fem.VectorFunctionSpace(domain, ("CG", 2))

INFO:root:running build_ext
INFO:root:building 'libffcx_elements_2d5c9a986741d359a5c76c1d0ce7466e4777b687' extension
INFO:root:x86_64-linux-gnu-gcc -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -I/usr/local/lib/python3.10/dist-packages/ffcx/codegeneration -I/usr/include/python3.10 -c libffcx_elements_2d5c9a986741d359a5c76c1d0ce7466e4777b687.c -o ./libffcx_elements_2d5c9a986741d359a5c76c1d0ce7466e4777b687.o -O2
INFO:root:x86_64-linux-gnu-gcc -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -g -fwrapv -O2 ./libffcx_elements_2d5c9a986741d359a5c76c1d0ce7466e4777b687.o -L/usr/lib/x86_64-linux-gnu -o ./libffcx_elements_2d5c9a986741d359a5c76c1d0ce7466e4777b687.cpython-310-x86_64-linux-gnu.so


We create two python functions for determining the facets to apply boundary conditions to

In [2]:
def left(x):
    return np.isclose(x[0], 0)

def right(x):
    return np.isclose(x[0], L)

fdim = domain.topology.dim -1
left_facets = mesh.locate_entities_boundary(domain, fdim, left)
right_facets = mesh.locate_entities_boundary(domain, fdim, right)

Next, we create a  marker based on these two functions

In [3]:
# Concatenate and sort the arrays based on facet indices. Left facets marked with 1, right facets with two
marked_facets = np.hstack([left_facets, right_facets])
marked_values = np.hstack([np.full_like(left_facets, 1), np.full_like(right_facets, 2)])
sorted_facets = np.argsort(marked_facets)
facet_tag = mesh.meshtags(domain, fdim, marked_facets[sorted_facets], marked_values[sorted_facets])

We then create a function for supplying the boundary condition on the left side, which is fixed.

In [4]:
u_bc = np.array((0,) * domain.geometry.dim, dtype=PETSc.ScalarType)

To apply the boundary condition, we identity the dofs located on the facets marked by the `MeshTag`.

In [5]:
left_dofs = fem.locate_dofs_topological(V, facet_tag.dim, facet_tag.find(1))
bcs = [fem.dirichletbc(u_bc, left_dofs, V)]

Next, we define the body force on the reference configuration (`B`), and nominal (first Piola-Kirchhoff) traction (`T`). 

In [6]:
B = fem.Constant(domain, PETSc.ScalarType((0, 0, 0)))
T = fem.Constant(domain, PETSc.ScalarType((0, 0, 0)))

Define the test and solution functions on the space $V$

In [7]:
v = ufl.TestFunction(V)
u = fem.Function(V)

Define kinematic quantities used in the problem

In [8]:
# Spatial dimension
d = len(u)

# Identity tensor
I = ufl.variable(ufl.Identity(d))

# Deformation gradient
F = ufl.variable(I + ufl.grad(u))

# Right Cauchy-Green tensor
C = ufl.variable(F.T * F)

# Invariants of deformation tensors
Ic = ufl.variable(ufl.tr(C))
J  = ufl.variable(ufl.det(F))

Define the elasticity model via a stored strain energy density function $\psi$, and create the expression for the first Piola-Kirchhoff stress:

In [9]:
# Elasticity parameters
E = PETSc.ScalarType(1.0e4)
nu = PETSc.ScalarType(0.3)
mu = fem.Constant(domain, E/(2*(1 + nu)))
lmbda = fem.Constant(domain, E*nu/((1 + nu)*(1 - 2*nu)))
# Stored strain energy density (compressible neo-Hookean model)
psi = (mu / 2) * (Ic - 3) - mu * ufl.ln(J) + (lmbda / 2) * (ufl.ln(J))**2
# Stress
# Hyper-elasticity
P = ufl.diff(psi, F)

```{admonition} Comparison to linear elasticity
To illustrate the difference between linear and hyperelasticity, the following lines can be uncommented to solve the linear elasticity problem.
```

In [10]:
# P = 2.0 * mu * ufl.sym(ufl.grad(u)) + lmbda * ufl.tr(ufl.sym(ufl.grad(u))) * I

Define the variational form with traction integral over all facets with value 2. We set the quadrature degree for the integrals to 4.

In [11]:
metadata = {"quadrature_degree": 4}
ds = ufl.Measure('ds', domain=domain, subdomain_data=facet_tag, metadata=metadata)
dx = ufl.Measure("dx", domain=domain, metadata=metadata)
# Define form F (we want to find u such that F(u) = 0)
F = ufl.inner(ufl.grad(v), P)*dx - ufl.inner(v, B)*dx - ufl.inner(v, T)*ds(2) 

As the varitional form is non-linear and written on residual form, we use the non-linear problem class from DOLFINx to set up required structures to use a Newton solver.

In [12]:
problem = fem.petsc.NonlinearProblem(F, u, bcs)

INFO:root:running build_ext
INFO:root:building 'libffcx_forms_69a876110b554279567433f944d3802b15088492' extension
INFO:root:x86_64-linux-gnu-gcc -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -I/usr/local/lib/python3.10/dist-packages/ffcx/codegeneration -I/usr/include/python3.10 -c libffcx_forms_69a876110b554279567433f944d3802b15088492.c -o ./libffcx_forms_69a876110b554279567433f944d3802b15088492.o -O2
INFO:root:x86_64-linux-gnu-gcc -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -g -fwrapv -O2 ./libffcx_forms_69a876110b554279567433f944d3802b15088492.o -L/usr/lib/x86_64-linux-gnu -o ./libffcx_forms_69a876110b554279567433f944d3802b15088492.cpython-310-x86_64-linux-gnu.so
INFO:root:running build_ext
INFO:root:building 'libffcx_forms_78b407f829a4cb0618fde7e20bfee058f65b12bd' extension
INFO:root:x86_64-linux-gnu-gcc -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -

and then create and customize the Newton solver

In [13]:
from dolfinx import nls
solver = nls.petsc.NewtonSolver(domain.comm, problem)

# Set Newton solver options
solver.atol = 1e-8
solver.rtol = 1e-8
solver.convergence_criterion = "incremental"


We create a function to plot the solution at each time step.

In [14]:
import pyvista
import matplotlib.pyplot as plt
pyvista.start_xvfb()
plotter = pyvista.Plotter()
plotter.open_gif("deformation.gif", fps=3)

topology, cells, geometry = plot.create_vtk_mesh(u.function_space)
function_grid = pyvista.UnstructuredGrid(topology, cells, geometry)

values = np.zeros((geometry.shape[0], 3))
values[:, :len(u)] = u.x.array.reshape(geometry.shape[0], len(u))
function_grid["u"] = values
function_grid.set_active_vectors("u")

# Warp mesh by deformation
warped = function_grid.warp_by_vector("u", factor=1)
warped.set_active_vectors("u")

# Add mesh to plotter and visualize
actor = plotter.add_mesh(warped, show_edges=True, lighting=False, clim=[0, 10])

# Compute magnitude of displacement to visualize in GIF
Vs = fem.FunctionSpace(domain, ("Lagrange", 2))
magnitude = fem.Function(Vs)
us = fem.Expression(ufl.sqrt(sum([u[i]**2 for i in range(len(u))])), Vs.element.interpolation_points())
magnitude.interpolate(us)
warped["mag"] = magnitude.x.array

INFO:root:running build_ext
INFO:root:building 'libffcx_elements_1779003a382381322b01b435c8795eb07e36a470' extension
INFO:root:x86_64-linux-gnu-gcc -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -I/usr/local/lib/python3.10/dist-packages/ffcx/codegeneration -I/usr/include/python3.10 -c libffcx_elements_1779003a382381322b01b435c8795eb07e36a470.c -o ./libffcx_elements_1779003a382381322b01b435c8795eb07e36a470.o -O2
INFO:root:x86_64-linux-gnu-gcc -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -g -fwrapv -O2 ./libffcx_elements_1779003a382381322b01b435c8795eb07e36a470.o -L/usr/lib/x86_64-linux-gnu -o ./libffcx_elements_1779003a382381322b01b435c8795eb07e36a470.cpython-310-x86_64-linux-gnu.so
INFO:root:running build_ext
INFO:root:building 'libffcx_expressions_1c636c4fefab7bb38773748e9689319cbb908186' extension
INFO:root:x86_64-linux-gnu-gcc -Wno-unused-result -Wsign-compare -DND

Finally, we solve the problem over several time steps, updating the y-component of the traction

In [15]:
from dolfinx import log
log.set_log_level(log.LogLevel.INFO)
tval0 = -1.5
for n in range(1, 10):
    T.value[2] = n * tval0
    num_its, converged = solver.solve(u)
    assert(converged)
    u.x.scatter_forward()
    print(f"Time step {n}, Number of iterations {num_its}, Load {T.value}")
    function_grid["u"][:, :len(u)] = u.x.array.reshape(geometry.shape[0], len(u))
    magnitude.interpolate(us)
    warped.set_active_scalars("mag")
    warped_n = function_grid.warp_by_vector(factor=1)
    plotter.update_coordinates(warped_n.points.copy(), render=False)
    plotter.update_scalar_bar_range([0, 10])
    plotter.update_scalars(magnitude.x.array)
    plotter.write_frame()
plotter.close()

2023-01-31 20:56:18.901 (  10.147s) [main            ]              petsc.cpp:675   INFO| PETSc Krylov solver starting to solve system.
2023-01-31 20:56:19.544 (  10.789s) [main            ]         TimeLogger.cpp:28    INFO| Elapsed wall, usr, sys time: 0.642597, 0.610000, 0.040000 (PETSc Krylov solver)
2023-01-31 20:56:19.784 (  11.029s) [main            ]              petsc.cpp:675   INFO| PETSc Krylov solver starting to solve system.
2023-01-31 20:56:20.238 (  11.484s) [main            ]         TimeLogger.cpp:28    INFO| Elapsed wall, usr, sys time: 0.454172, 0.430000, 0.030000 (PETSc Krylov solver)
2023-01-31 20:56:20.251 (  11.497s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 2: r (abs) = 22.2455 (tol = 1e-08) r (rel) = 0.134278(tol = 1e-08)
2023-01-31 20:56:20.478 (  11.724s) [main            ]              petsc.cpp:675   INFO| PETSc Krylov solver starting to solve system.
2023-01-31 20:56:20.939 (  12.185s) [main            ]         TimeLogger.cpp:

Time step 1, Number of iterations 8, Load [ 0.   0.  -1.5]


2023-01-31 20:56:24.763 (  16.008s) [main            ]              petsc.cpp:675   INFO| PETSc Krylov solver starting to solve system.
2023-01-31 20:56:25.152 (  16.397s) [main            ]         TimeLogger.cpp:28    INFO| Elapsed wall, usr, sys time: 0.388750, 0.350000, 0.040000 (PETSc Krylov solver)
2023-01-31 20:56:25.378 (  16.623s) [main            ]              petsc.cpp:675   INFO| PETSc Krylov solver starting to solve system.
2023-01-31 20:56:25.795 (  17.041s) [main            ]         TimeLogger.cpp:28    INFO| Elapsed wall, usr, sys time: 0.417631, 0.350000, 0.070000 (PETSc Krylov solver)
2023-01-31 20:56:25.805 (  17.050s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 2: r (abs) = 17.3254 (tol = 1e-08) r (rel) = 0.117842(tol = 1e-08)
2023-01-31 20:56:25.989 (  17.235s) [main            ]              petsc.cpp:675   INFO| PETSc Krylov solver starting to solve system.
2023-01-31 20:56:26.491 (  17.737s) [main            ]         TimeLogger.cpp:

Time step 2, Number of iterations 9, Load [ 0.  0. -3.]


2023-01-31 20:56:30.960 (  22.206s) [main            ]              petsc.cpp:675   INFO| PETSc Krylov solver starting to solve system.
2023-01-31 20:56:31.375 (  22.621s) [main            ]         TimeLogger.cpp:28    INFO| Elapsed wall, usr, sys time: 0.414935, 0.360000, 0.050000 (PETSc Krylov solver)
2023-01-31 20:56:31.641 (  22.886s) [main            ]              petsc.cpp:675   INFO| PETSc Krylov solver starting to solve system.
2023-01-31 20:56:32.085 (  23.331s) [main            ]         TimeLogger.cpp:28    INFO| Elapsed wall, usr, sys time: 0.444717, 0.400000, 0.050000 (PETSc Krylov solver)
2023-01-31 20:56:32.108 (  23.353s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 2: r (abs) = 10.0011 (tol = 1e-08) r (rel) = 0.0887471(tol = 1e-08)
2023-01-31 20:56:32.367 (  23.612s) [main            ]              petsc.cpp:675   INFO| PETSc Krylov solver starting to solve system.
2023-01-31 20:56:32.798 (  24.044s) [main            ]         TimeLogger.cpp

...

2023-01-31 20:56:59.574 (  50.820s) [main            ]              petsc.cpp:675   INFO| PETSc Krylov solver starting to solve system.
2023-01-31 20:57:00.034 (  51.279s) [main            ]         TimeLogger.cpp:28    INFO| Elapsed wall, usr, sys time: 0.459883, 0.420000, 0.040000 (PETSc Krylov solver)
2023-01-31 20:57:00.329 (  51.574s) [main            ]              petsc.cpp:675   INFO| PETSc Krylov solver starting to solve system.
2023-01-31 20:57:00.842 (  52.088s) [main            ]         TimeLogger.cpp:28    INFO| Elapsed wall, usr, sys time: 0.513549, 0.480000, 0.030000 (PETSc Krylov solver)
2023-01-31 20:57:00.852 (  52.097s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 2: r (abs) = 1.06336 (tol = 1e-08) r (rel) = 0.031085(tol = 1e-08)
2023-01-31 20:57:01.098 (  52.344s) [main            ]              petsc.cpp:675   INFO| PETSc Krylov solver starting to solve system.
2023-01-31 20:57:01.615 (  52.861s) [main            ]         TimeLogger.cpp:

Time step 8, Number of iterations 6, Load [  0.   0. -12.]


2023-01-31 20:57:04.236 (  55.481s) [main            ]              petsc.cpp:675   INFO| PETSc Krylov solver starting to solve system.
2023-01-31 20:57:04.695 (  55.941s) [main            ]         TimeLogger.cpp:28    INFO| Elapsed wall, usr, sys time: 0.459318, 0.410000, 0.050000 (PETSc Krylov solver)
2023-01-31 20:57:04.965 (  56.211s) [main            ]              petsc.cpp:675   INFO| PETSc Krylov solver starting to solve system.
2023-01-31 20:57:05.415 (  56.661s) [main            ]         TimeLogger.cpp:28    INFO| Elapsed wall, usr, sys time: 0.450075, 0.420000, 0.040000 (PETSc Krylov solver)
2023-01-31 20:57:05.425 (  56.671s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 2: r (abs) = 0.898789 (tol = 1e-08) r (rel) = 0.0309666(tol = 1e-08)
2023-01-31 20:57:05.649 (  56.895s) [main            ]              petsc.cpp:675   INFO| PETSc Krylov solver starting to solve system.
2023-01-31 20:57:06.129 (  57.375s) [main            ]         TimeLogger.cp

Time step 9, Number of iterations 6, Load [  0.    0.  -13.5]


2023-01-31 20:57:07.791 (  59.037s) [main            ]              petsc.cpp:675   INFO| PETSc Krylov solver starting to solve system.
2023-01-31 20:57:08.299 (  59.544s) [main            ]         TimeLogger.cpp:28    INFO| Elapsed wall, usr, sys time: 0.507236, 0.450000, 0.050000 (PETSc Krylov solver)
2023-01-31 20:57:08.312 (  59.558s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 6: r (abs) = 2.81885e-13 (tol = 1e-08) r (rel) = 9.71198e-15(tol = 1e-08)
2023-01-31 20:57:08.312 (  59.558s) [main            ]       NewtonSolver.cpp:255   INFO| Newton solver finished in 6 iterations and 6 linear solver iterations.


<img src="./deformation.gif" alt="gif" class="bg-primary mb-1" width="800px">